In [1]:
%matplotlib inline

In [2]:
!pip install pydub torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
!pip install datasets[audio]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# !wget -P '/content/drive/MyDrive/18781 - Speech Recognition Project/' 'https://s3.amazonaws.com/datasets.huggingface.co/SpeechCommands/v0.02/v0.02_validation.tar.gz'
# !wget -P '/content/drive/MyDrive/18781 - Speech Recognition Project/' 'https://s3.amazonaws.com/datasets.huggingface.co/SpeechCommands/v0.02/v0.02_train.tar.gz'
# !wget -P '/content/drive/MyDrive/18781 - Speech Recognition Project/' 'https://s3.amazonaws.com/datasets.huggingface.co/SpeechCommands/v0.02/v0.02_test.tar.gz'


##Dataset

In [6]:
!mkdir train
!mkdir validation
!mkdir test

!tar -xf '/content/drive/MyDrive/18781 - Speech Recognition Project/v0.02_train.tar.gz' -C './train'
!tar -xf '/content/drive/MyDrive/18781 - Speech Recognition Project/v0.02_validation.tar.gz' -C './validation'
!tar -xf '/content/drive/MyDrive/18781 - Speech Recognition Project/v0.02_test.tar.gz' -C './test'

In [7]:
!apt -qq install -y sox

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 5 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack 

###Train Dataset

In [8]:
!cd ./train/_silence_/ ; sox pink_noise.wav pink_noise.wav trim 0 1 : newfile : restart ; rm -rf pink_noise.wav
!cd ./train/_silence_/ ; sox doing_the_dishes.wav doing_the_dishes.wav trim 0 1 : newfile : restart ; rm -rf doing_the_dishes.wav
!cd ./train/_silence_/ ; sox exercise_bike.wav exercise_bike.wav trim 0 1 : newfile : restart ; rm -rf exercise_bike.wav
!cd ./train/_silence_/ ; sox dude_miaowing.wav dude_miaowing.wav trim 0 1 : newfile : restart ; rm -rf dude_miaowing.wav
!cd ./train/_silence_/ ; sox white_noise.wav white_noise.wav trim 0 1 : newfile : restart ; rm -rf white_noise.wav

!cd ./train ; mkdir _not_silence_
!cd ./train ; cp backward/* _not_silence_/ ; rm -rf backward
!cd ./train ; cp bed/* _not_silence_/ ; rm -rf bed
!cd ./train ; cp bird/* _not_silence_/ ; rm -rf bird
!cd ./train ; cp cat/* _not_silence_/ ; rm -rf cat
!cd ./train ; cp dog/* _not_silence_/ ; rm -rf dog
!cd ./train ; cp down/* _not_silence_/ ; rm -rf down
!cd ./train ; cp eight/* _not_silence_/ ; rm -rf eight
!cd ./train ; cp five/* _not_silence_/ ; rm -rf five
!cd ./train ; cp follow/* _not_silence_/ ; rm -rf follow
!cd ./train ; cp forward/* _not_silence_/ ; rm -rf forward
!cd ./train ; cp four/* _not_silence_/ ; rm -rf four
!cd ./train ; cp go/* _not_silence_/ ; rm -rf go
!cd ./train ; cp happy/* _not_silence_/ ; rm -rf happy
!cd ./train ; cp house/* _not_silence_/ ; rm -rf house
!cd ./train ; cp learn/* _not_silence_/ ; rm -rf learn
!cd ./train ; cp left/* _not_silence_/ ; rm -rf left
!cd ./train ; cp marvin/* _not_silence_/ ; rm -rf marvin
!cd ./train ; cp nine/* _not_silence_/ ; rm -rf nine
!cd ./train ; cp no/* _not_silence_/ ; rm -rf no
!cd ./train ; cp off/* _not_silence_/ ; rm -rf off
!cd ./train ; cp on/* _not_silence_/ ; rm -rf on
!cd ./train ; cp one/* _not_silence_/ ; rm -rf one
!cd ./train ; cp right/* _not_silence_/ ; rm -rf right
!cd ./train ; cp seven/* _not_silence_/ ; rm -rf seven
!cd ./train ; cp sheila/* _not_silence_/ ; rm -rf sheila
!cd ./train ; cp six/* _not_silence_/ ; rm -rf six
!cd ./train ; cp stop/* _not_silence_/ ; rm -rf stop
!cd ./train ; cp three/* _not_silence_/ ; rm -rf three
!cd ./train ; cp tree/* _not_silence_/ ; rm -rf tree
!cd ./train ; cp two/* _not_silence_/ ; rm -rf two
!cd ./train ; cp up/* _not_silence_/ ; rm -rf up
!cd ./train ; cp visual/* _not_silence_/ ; rm -rf visual
!cd ./train ; cp wow/* _not_silence_/ ; rm -rf wow
!cd ./train ; cp yes/* _not_silence_/ ; rm -rf yes
!cd ./train ; cp zero/* _not_silence_/ ; rm -rf zero

sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).
sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).
sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).


In [5]:
from datasets import load_dataset, Audio

train = load_dataset(path='/content/train')
train = train.cast_column("audio", Audio(sampling_rate=8000))

Resolving data files:   0%|          | 0/7473 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

###Test Dataset

In [11]:
!cd ./validation/_silence_/ ; sox running_tap.wav running_tap.wav trim 0 1 : newfile : restart ; rm -rf running_tap.wav

!cd ./validation ; mkdir _not_silence_
!cd ./validation ; cp backward/* _not_silence_/ ; rm -rf backward
!cd ./validation ; cp bed/* _not_silence_/ ; rm -rf bed
!cd ./validation ; cp bird/* _not_silence_/ ; rm -rf bird
!cd ./validation ; cp cat/* _not_silence_/ ; rm -rf cat
!cd ./validation ; cp dog/* _not_silence_/ ; rm -rf dog
!cd ./validation ; cp down/* _not_silence_/ ; rm -rf down
!cd ./validation ; cp eight/* _not_silence_/ ; rm -rf eight
!cd ./validation ; cp five/* _not_silence_/ ; rm -rf five
!cd ./validation ; cp follow/* _not_silence_/ ; rm -rf follow
!cd ./validation ; cp forward/* _not_silence_/ ; rm -rf forward
!cd ./validation ; cp four/* _not_silence_/ ; rm -rf four
!cd ./validation ; cp go/* _not_silence_/ ; rm -rf go
!cd ./validation ; cp happy/* _not_silence_/ ; rm -rf happy
!cd ./validation ; cp house/* _not_silence_/ ; rm -rf house
!cd ./validation ; cp learn/* _not_silence_/ ; rm -rf learn
!cd ./validation ; cp left/* _not_silence_/ ; rm -rf left
!cd ./validation ; cp marvin/* _not_silence_/ ; rm -rf marvin
!cd ./validation ; cp nine/* _not_silence_/ ; rm -rf nine
!cd ./validation ; cp no/* _not_silence_/ ; rm -rf no
!cd ./validation ; cp off/* _not_silence_/ ; rm -rf off
!cd ./validation ; cp on/* _not_silence_/ ; rm -rf on
!cd ./validation ; cp one/* _not_silence_/ ; rm -rf one
!cd ./validation ; cp right/* _not_silence_/ ; rm -rf right
!cd ./validation ; cp seven/* _not_silence_/ ; rm -rf seven
!cd ./validation ; cp sheila/* _not_silence_/ ; rm -rf sheila
!cd ./validation ; cp six/* _not_silence_/ ; rm -rf six
!cd ./validation ; cp stop/* _not_silence_/ ; rm -rf stop
!cd ./validation ; cp three/* _not_silence_/ ; rm -rf three
!cd ./validation ; cp tree/* _not_silence_/ ; rm -rf tree
!cd ./validation ; cp two/* _not_silence_/ ; rm -rf two
!cd ./validation ; cp up/* _not_silence_/ ; rm -rf up
!cd ./validation ; cp visual/* _not_silence_/ ; rm -rf visual
!cd ./validation ; cp wow/* _not_silence_/ ; rm -rf wow
!cd ./validation ; cp yes/* _not_silence_/ ; rm -rf yes
!cd ./validation ; cp zero/* _not_silence_/ ; rm -rf zero

sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).


In [12]:
!cd ./test ; mkdir _not_silence_
!cd ./test ; cp backward/* _not_silence_/ ; rm -rf backward
!cd ./test ; cp bed/* _not_silence_/ ; rm -rf bed
!cd ./test ; cp bird/* _not_silence_/ ; rm -rf bird
!cd ./test ; cp cat/* _not_silence_/ ; rm -rf cat
!cd ./test ; cp dog/* _not_silence_/ ; rm -rf dog
!cd ./test ; cp down/* _not_silence_/ ; rm -rf down
!cd ./test ; cp eight/* _not_silence_/ ; rm -rf eight
!cd ./test ; cp five/* _not_silence_/ ; rm -rf five
!cd ./test ; cp follow/* _not_silence_/ ; rm -rf follow
!cd ./test ; cp forward/* _not_silence_/ ; rm -rf forward
!cd ./test ; cp four/* _not_silence_/ ; rm -rf four
!cd ./test ; cp go/* _not_silence_/ ; rm -rf go
!cd ./test ; cp happy/* _not_silence_/ ; rm -rf happy
!cd ./test ; cp house/* _not_silence_/ ; rm -rf house
!cd ./test ; cp learn/* _not_silence_/ ; rm -rf learn
!cd ./test ; cp left/* _not_silence_/ ; rm -rf left
!cd ./test ; cp marvin/* _not_silence_/ ; rm -rf marvin
!cd ./test ; cp nine/* _not_silence_/ ; rm -rf nine
!cd ./test ; cp no/* _not_silence_/ ; rm -rf no
!cd ./test ; cp off/* _not_silence_/ ; rm -rf off
!cd ./test ; cp on/* _not_silence_/ ; rm -rf on
!cd ./test ; cp one/* _not_silence_/ ; rm -rf one
!cd ./test ; cp right/* _not_silence_/ ; rm -rf right
!cd ./test ; cp seven/* _not_silence_/ ; rm -rf seven
!cd ./test ; cp sheila/* _not_silence_/ ; rm -rf sheila
!cd ./test ; cp six/* _not_silence_/ ; rm -rf six
!cd ./test ; cp stop/* _not_silence_/ ; rm -rf stop
!cd ./test ; cp three/* _not_silence_/ ; rm -rf three
!cd ./test ; cp tree/* _not_silence_/ ; rm -rf tree
!cd ./test ; cp two/* _not_silence_/ ; rm -rf two
!cd ./test ; cp up/* _not_silence_/ ; rm -rf up
!cd ./test ; cp visual/* _not_silence_/ ; rm -rf visual
!cd ./test ; cp wow/* _not_silence_/ ; rm -rf wow
!cd ./test ; cp yes/* _not_silence_/ ; rm -rf yes
!cd ./test ; cp zero/* _not_silence_/ ; rm -rf zero

In [13]:
!mv validation/_silence_/* test/_silence_/
!mv validation/_not_silence_/* test/_not_silence_/ ; rm -rf validation

In [6]:
test = load_dataset(path='/content/test')
test = test.cast_column("audio", Audio(sampling_rate=8000))

Resolving data files:   0%|          | 0/2064 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
train_set = train['train']
test_set = test['train']

In [8]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    batch = torch.unsqueeze(batch, dim=1)
    return batch


def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for i in batch:
      tensors += [torch.from_numpy(i['audio']['array'])]
      targets += [torch.tensor(i['label'])]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets

In [31]:
batch_size = 32

In [32]:
if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

In [33]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=2, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return x


model = M5(n_input=1, n_output=2)
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=2, bias=True)
)
Number

In [34]:
# zero_count = 0
# one_count = 1

# for datapoint in train_set:
#   if datapoint['label']==0:
#     zero_count +=1
#   elif datapoint['label']==1:
#     one_count +=1

In [35]:
# print("0 :", zero_count, "1 :", one_count)

# total = (zero_count + one_count)
# weight_vector = torch.tensor([1, zero_count/one_count])

weight_vector = torch.tensor([1,21]).cuda()
weight_vector

tensor([ 1, 21], device='cuda:0')

In [36]:

optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10
loss_fn = nn.BCEWithLogitsLoss(pos_weight=weight_vector)

In [37]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        # data = transform(data)
        output = model(data)


        # print(output.shape, target.shape)
        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = nn.CrossEntropyLoss(reduction = "none")(output.squeeze(), target.long())
        loss[target == 1] *= 21
        loss = loss.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [38]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


In [39]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

In [40]:
log_interval = 20
n_epoch = 20

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)
        scheduler.step()

# Let's plot the training loss versus the number of iteration.
# plt.plot(losses);
# plt.title("training loss");

  0%|          | 0.0033444816053511705/20 [00:00<49:50, 149.55s/it]

Train Epoch: 1 [0/7473 (0%)]	Loss: 0.567645


  0%|          | 0.07023411371237456/20 [00:10<47:23, 142.68s/it]

Train Epoch: 1 [640/7473 (9%)]	Loss: 1.207877


  1%|          | 0.13712374581939793/20 [00:19<46:39, 140.93s/it]

...

Train Epoch: 20 [6400/7473 (85%)]	Loss: 0.015720


 99%|█████████▊| 19.739130434781785/20 [47:40<00:35, 136.79s/it]

Train Epoch: 20 [7040/7473 (94%)]	Loss: 0.111717


100%|█████████▉| 19.999999999999055/20 [48:17<00:00, 144.88s/it]


Test Epoch: 20	Accuracy: 1986/2064 (96%)



In [56]:
def predict(tensor):
  # Use the model to predict the label of the waveform
  tensor = tensor.to(device)
  tensor = model(tensor.unsqueeze(0))
  tensor = get_likely_index(tensor)
  return tensor

datapoint = train_set[45]

print(f"Expected: ,", datapoint["label"], ". Predicted: ",predict(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)),".")

Expected: , 0 . Predicted:  tensor([[0]], device='cuda:0') .
